# Creating a Batch Job to Dynamically Load the Champion Model

So far we've seen how we can interact with the Model Version Sets to label and re-label our models in order to version control and manage the model lifecycle. 

In the below we'll see how we can create an OCI Data Science Job that automatically loads the current model champion as part of the workflow. For a production environment, this could be useful where ModelOps processes periodically retrain and replace the current champion and where batch jobs run on a regular cadence.

### Load Dependencies

In [1]:
from ads.jobs import Job, DataScienceJob, ScriptRuntime
import ads
from ads.common.auth import default_signer

### Set Auth and OCIDs

In [2]:
ads.set_auth(auth='api_key')

In [ ]:
# set OCIDs

log_group_id = 'ocid1.loggroup...'
compartment_ocid = 'ocid1.compartment...'
project_ocid = 'ocid1.datascienceproject...'
conda_env = 'automlx251_p311_cpu_x86_64_v2'
source_dir = 'score_champion_model.py'

### Define Job

In [12]:
job = (
    Job(name="Score Current Champion")
    .with_infrastructure(
        DataScienceJob()
        # Configure logging for getting the job run outputs.
        .with_log_group_id(log_group_id)
        # If you are in an OCI data science notebook session,
        # the following configurations are not required.
        # Configurations from the notebook session will be used as defaults.
        .with_compartment_id(compartment_ocid)
        .with_project_id(project_ocid)
        #.with_subnet_id("<subnet_ocid>") # use service managed egress
        .with_shape_name("VM.Standard.E4.Flex")
        # Shape config details are applicable only for the flexible shapes.
        .with_shape_config_details(memory_in_gbs=16, ocpus=1)
        # Minimum/Default block storage size is 50 (GB).
        .with_block_storage_size(50)
    )
    .with_runtime(
        ScriptRuntime()
        # Specify the service conda environment by slug name.
        .with_service_conda(conda_env)
        # The job artifact can be a single Python script, a directory or a zip file.
        .with_source(source_dir)
    )
)

### Create Job

In [ ]:
job.create()

### Run and Monitor Job

In [ ]:
run = job.run('Score current champion')

run.watch()